## Approximate Functional Dependency Discovery and Validation

In [16]:
from typing import Tuple, List, Dict, Any
from dataclasses import dataclass, field

import pandas as pd
import desbordante
import desbordante.afd.algorithms as afd_algorithms

from pprint import pprint

In [17]:
file_path = "../../data/Food_Inspections_20250216_preprocessed.parquet"
df = pd.read_parquet(file_path)
df.head()

,dba_name,license_,facility_type,risk,address,city,zip,inspection_date,results,violations,latitude,longitude
0,BLOOMING BUD DAYCARE,2215789,Daycare Combo 1586,Risk 1 (High),5715 N LINCOLN AVE,CHICAGO,60659,03/07/2013,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.985390,-87.698734
1,Babas Halal,2684170,Restaurant,Risk 1 (High),7901 S DAMEN AVE,CHICAGO,60620,12/03/2024,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.750189,-87.672986
2,FIRST ZABIHA MEAT BAZAAR,2232559,Grocery Store,Risk 2 (Medium),2907 W DEVON AVE,CHICAGO,60659,02/20/2013,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.997401,-87.702385
3,HAPPY MARKET,2912802,Grocery Store,Risk 2 (Medium),2334 S WENTWORTH AVE,CHICAGO,60616,01/02/2025,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.849954,-87.632094
4,SAT KAIVAL FOOD INC/SUBWAY,2728400,Restaurant,Risk 1 (High),1916 S STATE ST,CHICAGO,60616,01/02/2025,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.856053,-87.627311


In [18]:
def find_afds(df:pd.DataFrame, algorithm_name:str='Default', error:float=0.1):
    """
    Finds approximate functional dependencies in a given DataFrame using a specified algorithm.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        algorithm_name (str): The name of the FD algorithm to use. Defaults to 'Default'.
    
    Returns:
        list: A list of discovered approximate functional dependencies.
    """
    try:
        # Get the algorithm class dynamically from desbordante.fd.algorithms
        algo_class = getattr(afd_algorithms, algorithm_name, afd_algorithms.Default)

        print(f"Algorthm: {algo_class}")
        
        algo = algo_class()
        algo.load_data(table=df)
        algo.execute(error=error)
        return algo.get_fds()
    except AttributeError:
        raise ValueError(f"Algorithm '{algorithm_name}' not found. Available algorithms: {dir(afd_algorithms)}")

In [19]:
@dataclass
class FunctionalDependency:
    lhs: List[str]  # Left-hand side attributes
    rhs: str        # Right-hand side attribute

    def __str__(self):
       lhs_count = len(self.lhs)
       base = f"LHS={self.lhs} ({lhs_count}), RHS={self.rhs}"
       return base
    
@dataclass
class FunctionalDependencySet:
    dependencies: List[FunctionalDependency] = field(default_factory=list)
    validation_results: Dict[Tuple[Tuple[str, ...], str], Dict[str, Any]] = field(default_factory=dict)


    def add_dependency(self, lhs: List[str], rhs: str):
        """Adds a new functional dependency to the set."""
        self.dependencies.append(FunctionalDependency(lhs, rhs))

    def __len__(self):
        """Returns the number of functional dependencies."""
        return len(self.dependencies)

    def __iter__(self):
        """Allows iteration over functional dependencies."""
        return iter(self.dependencies)
    
    def validate_afd_dependencies(self, df:pd.DataFrame, error:float=0.05):
        """Validates all functional dependencies in the dataset and stores the results."""
        GREEN_CODE = "\033[1;42m"
        RED_CODE = "\033[1;41m"
        DEFAULT_COLOR_CODE = "\033[1;49m"

        verifier = desbordante.afd_verification.algorithms.Default()
            
        verifier.load_data(table=df)

        for fd in self.dependencies:
            lhs_idx = df.columns.get_indexer(fd.lhs)
            rhs_idx = df.columns.get_loc(fd.rhs)

            if lhs_idx[0] == -1:
                continue
            
            verifier.execute(lhs_indices=lhs_idx, rhs_indices=[rhs_idx])
            highlights = verifier.get_highlights()

            fd_holds = verifier.get_error() < error

            if fd_holds:
                print(GREEN_CODE, "AFD with this error threshold holds", DEFAULT_COLOR_CODE)
            else:
                print(RED_CODE, "AFD with this error threshold does not hold", DEFAULT_COLOR_CODE)
                print(f"But the same {GREEN_CODE} AFD with error threshold = {verifier.get_error()} holds{DEFAULT_COLOR_CODE}")


            fd_key = (tuple(fd.lhs), fd.rhs)
            self.validation_results[fd_key] = {
                "holds": fd_holds,
                "num_violations": verifier.get_num_error_clusters(),
                "highlights": highlights
            }

            if self.validation_results[fd_key]["holds"]:
                print(GREEN_CODE, f"FD holds: {fd.lhs} -> {fd.rhs}", DEFAULT_COLOR_CODE)
            else:
                print(RED_CODE, f"FD does not hold: {fd.lhs} -> {fd.rhs}", DEFAULT_COLOR_CODE)
                print(f"Number of clusters violating FD: {self.validation_results[fd_key]['num_violations']}")

    def get_validation_result(self, lhs: List[str], rhs: str) -> Dict[str, Any]:
        """Retrieves stored validation results for a specific FD."""
        fd_key = (tuple(lhs), rhs)
        return self.validation_results.get(fd_key, {})

    def get_all_validation_results(self) -> Dict[Tuple[List, str], Dict[str, Any]]:
        """Returns all stored validation results."""
        return self.validation_results


In [20]:
def convert_fd(fd:desbordante.fd.FD) -> Tuple[list, str]:
    fd_str = str(fd) # convert fd to string
    fd_str_split = fd_str.split("->") # split fd to lhs and rhs
    lhs = fd_str_split[0].strip() 
    rhs = fd_str_split[-1].strip()

    lhs_list = lhs[1:-1].split(' ') # convert lhs to list of attributes

    return lhs_list, rhs

In [21]:
results = find_afds(df, error=0.05)

print(f"There are {len(results)} functional dependencies using Default algorithm.")

for fd in results:
    print(fd)

Algorthm: <class 'desbordante.fd.algorithms.Pyro'>
There are 97 functional dependencies using Default algorithm.
[] -> city
[dba_name] -> risk
[address] -> longitude
[license_] -> zip
[license_] -> risk
[latitude] -> longitude
[license_] -> results
[license_] -> dba_name
[license_] -> violations
[address] -> zip
[address] -> risk
[address] -> latitude
[longitude] -> latitude
[address] -> violations
[address] -> dba_name
[address] -> results
[inspection_date] -> risk
[dba_name] -> facility_type
[inspection_date] -> violations
[license_] -> address
[latitude] -> violations
[latitude] -> dba_name
[address] -> license_
[latitude] -> zip
[latitude] -> results
[latitude] -> risk
[latitude] -> address
[license_] -> longitude
[license_] -> facility_type
[longitude] -> risk
[longitude] -> violations
[license_] -> latitude
[zip] -> risk
[violations] -> risk
[longitude] -> results
[longitude] -> zip
[longitude] -> address
[longitude] -> dba_name
[address] -> facility_type
[dba_name] -> violations

In [22]:
fd_set = FunctionalDependencySet()
for result in results:
    lhs, rhs =  convert_fd(fd=result)
    fd_set.add_dependency(lhs, rhs)

In [23]:
# Validate all dependencies and store results
fd_set.validate_afd_dependencies(df, error=0.05)

 AFD with this error threshold holds 
 FD holds: ['dba_name'] -> risk 
 AFD with this error threshold holds 
 FD holds: ['address'] -> longitude 
 AFD with this error threshold holds 
 FD holds: ['license_'] -> zip 
 AFD with this error threshold holds 
 FD holds: ['license_'] -> risk 
 AFD with this error threshold holds 
 FD holds: ['latitude'] -> longitude 
 AFD with this error threshold holds 
 FD holds: ['license_'] -> results 
 AFD with this error threshold holds 
 FD holds: ['license_'] -> dba_name 
 AFD with this error threshold holds 
 FD holds: ['license_'] -> violations 
 AFD with this error threshold holds 
 FD holds: ['address'] -> zip 
 AFD with this error threshold holds 
 FD holds: ['address'] -> risk 
 AFD with this error threshold holds 
 FD holds: ['address'] -> latitude 
 AFD with this error threshold holds 
 FD holds: ['longitude'] -> latitude 
 AFD with this error threshold holds 
 FD holds: ['address'] -> violations 
 AFD with this error threshold holds 
 FD hold

In [24]:
print(f"There are {len(fd_set)} fds in the dataset.")

There are 97 fds in the dataset.


In [25]:
# Retrieve validation result for a specific FD
result = fd_set.get_validation_result(lhs=['risk', 'violations'],rhs="license_")

print(f"FD holds: {result['holds']}. There are {result['num_violations']} violations.")

FD holds: True. There are 627 violations.


In [26]:
all_results = fd_set.get_all_validation_results()

for (lhs, rhs), result in all_results.items():
    # Create a copy of result without the 'highlights' key
    filtered_result = {key: value for key, value in result.items() if key != "highlights"}
    
    print(f"FD: {lhs} -> {rhs}, Results: {filtered_result}")


FD: ('dba_name',) -> risk, Results: {'holds': True, 'num_violations': 1708}
FD: ('address',) -> longitude, Results: {'holds': True, 'num_violations': 0}
FD: ('license_',) -> zip, Results: {'holds': True, 'num_violations': 96}
FD: ('license_',) -> risk, Results: {'holds': True, 'num_violations': 271}
FD: ('latitude',) -> longitude, Results: {'holds': True, 'num_violations': 0}
FD: ('license_',) -> results, Results: {'holds': True, 'num_violations': 33083}
FD: ('license_',) -> dba_name, Results: {'holds': True, 'num_violations': 1163}
FD: ('license_',) -> violations, Results: {'holds': True, 'num_violations': 31629}
FD: ('address',) -> zip, Results: {'holds': True, 'num_violations': 50}
FD: ('address',) -> risk, Results: {'holds': True, 'num_violations': 4621}
FD: ('address',) -> latitude, Results: {'holds': True, 'num_violations': 0}
FD: ('longitude',) -> latitude, Results: {'holds': True, 'num_violations': 0}
FD: ('address',) -> violations, Results: {'holds': True, 'num_violations': 24